In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/NBChealth.txt'
dest_url = './run_data/nbc_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,548617649010532353,Fri Dec 26 23:13:35 +0000 2014,Ebola Exposure?: CDC Worker 'Remains Well' htt...
1,548569476795731968,Fri Dec 26 20:02:09 +0000 2014,Pregnant Woman Taken Off Life Support in Irela...
2,548533428023738368,Fri Dec 26 17:38:55 +0000 2014,Money from #IceBucketChallenge still flowing t...
3,548531135442337793,Fri Dec 26 17:29:48 +0000 2014,Caramel Apples Recalled After Listeria Deaths ...
4,548194819907284992,Thu Dec 25 19:13:24 +0000 2014,Ice Bucket Challenge Still Rains Bucks on ALS ...


In [3]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4215 entries, 0 to 4214
Data columns (total 3 columns):
tweetid    4215 non-null int64
date       4215 non-null object
title      4215 non-null object
dtypes: int64(1), object(2)
memory usage: 98.9+ KB


In [4]:
# -------------------------------------------------
frame.shape

(4215, 3)

In [5]:
# -------------------------------------------------
frame.loc[:,'title'][0]

"Ebola Exposure?: CDC Worker 'Remains Well' http://nbcnews.to/1tmYMkv"

In [6]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://nbcnews.to/1tmYMkv
http://nbcnews.to/1BcYvPG
http://ow.ly/GseY3
http://nbcnews.to/1EqpPjj
http://nbcnews.to/1thdMAd
http://ow.ly/GpzmS
http://nbcnews.to/1CLABvX
http://nbcnews.to/1vbYiYB
http://nbcnews.to/1EdtTDx
http://nbcnews.to/1zgmtY8
http://nbcnews.to/1CDgVKG
http://nbcnews.to/1zuy6Qa
http://nbcnews.to/13BkCEA
http://nbcnews.to/16NjUWM
http://nbcnews.to/1ztxbzv
http://nbcnews.to/1ADCoE6
http://nbcnews.to/1B17DXv
http://nbcnews.to/1B0cRTp
http://nbcnews.to/1t22uzR
http://nbcnews.to/1znY0Fm
http://nbcnews.to/1AMM1yo
http://nbcnews.to/1x90YgE
http://nbcnews.to/13rkdob
http://nbcnews.to/1z8qKg6
http://nbcnews.to/1CncO5i
http://nbcnews.to/1AqCGOg
http://nbcnews.to/1wOCXJS
http://nbcnews.to/1wyA1kF
http://nbcnews.to/1C9CKEm
http://nbcnews.to/1Gz4YrQ
http://nbcnews.to/1zFqgku
http://nbcnews.to/1H04M3c
http://nbcnews.to/1zCSIoa
http://nbcnews.to/1zCSJIP
http://nbcnews.to/1GvQOYp
http://nbcnews.to/13fPjPk
http://nbcnews.to/13eeS3x
http://nbcnews.to/1zxmFF4
http://nbcnews.to/1w0qaPx


http://nbcnews.to/1gcuTf0
http://nbcnews.to/1hDt2wp
http://nbcnews.to/1eninXJ
http://nbcnews.to/1ii0OsK
http://nbcnews.to/1gRiPLd
http://nbcnews.to/1lP8Qdv
http://nbcnews.to/OBGR5H
http://nbcnews.to/1ekORSA
http://nbcnews.to/1eRvFbL
http://nbcnews.to/1lNckNR
http://nbcnews.to/1eQyXfr
http://nbcnews.to/PNkD1S
http://nbcnews.to/1gMlmGv
http://nbcnews.to/1ifQ1Q1
http://nbcnews.to/1iZHEvm
http://nbcnews.to/1j17lqU
http://nbcnews.to/1l0vlik
http://nbcnews.to/1g6JGYx
http://nbcnews.to/1iZ8aos
http://nbcnews.to/Os5z8U
http://nbcnews.to/OxCCby
http://nbcnews.to/1gHiTNg
http://nbcnews.to/1gHiTwM
http://nbcnews.to/1p5ssKu
http://nbcnews.to/OwtOmc
http://nbcnews.to/1gvVXjB
http://nbcnews.to/1gvVXjv
http://nbcnews.to/OwtC6w
http://nbcnews.to/1oiaK8c
http://nbcnews.to/1oiaHcx
http://nbcnews.to/1iVeugP
http://nbcnews.to/1oiaHZZ
http://nbcnews.to/1nxEilg
http://nbcnews.to/1nwCDw6
http://nbcnews.to/1gomqze
http://nbcnews.to/1o6FqsP
http://nbcnews.to/NeN9HJ
http://nbcnews.to/1iKgqsp
http://nbcnews.to/1

http://nbcnews.to/1gNN3nV
http://Healthcare.gov
http://nbcnews.to/1aJYFQ4
http://nbcnews.to/1gNY60o
http://nbcnews.to/1aKyUPF
http://nbcnews.to/1gNQScA
http://nbcnews.to/1aJRirW
http://nbcnews.to/1gMMU40
http://nbcnews.to/1gNN3nV
http://Healthcare.gov
http://nbcnews.to/1aJYFQ4
http://nbcnews.to/1aKyUPF
http://nbcnews.to/1gNQScA
http://nbcnews.to/1gNY60o
http://nbcnews.to/1aJRirW
http://nbcnews.to/1gMMU40
http://nbcnews.to/1gNN3nV
http://Healthcare.gov
http://nbcnews.to/1aJYFQ4
http://nbcnews.to/1gNY60o
http://nbcnews.to/1aKyUPF
http://nbcnews.to/1gNQScA
http://nbcnews.to/1aJRirW
http://nbcnews.to/1gMMU40
http://Healthcare.gov
http://nbcnews.to/1aJYFQ4
http://nbcnews.to/1gNY60o
http://nbcnews.to/1gNQScA
http://nbcnews.to/1gNN3nV
http://nbcnews.to/1aKyUPF
http://nbcnews.to/1aJRirW
http://nbcnews.to/1gMMU40
http://nbcnews.to/1gNN3nV
http://Healthcare.gov
http://nbcnews.to/1aJYFQ4
http://nbcnews.to/1gNY60o
http://nbcnews.to/1aKyUPF
http://nbcnews.to/1gNQScA
http://nbcnews.to/1aJRirW
http:/

http://on.today.com/13Y2Xlh
http://on.today.com/18p1Tuk
http://nbcnews.to/18oGTUu
http://nbcnews.to/Yo2ecF
http://nbcnews.to/YnZCvt
http://nbcnews.to/YnWxLK
http://nbcnews.to/YlyexV
http://nbcnews.to/Yly2yP
http://nbcnews.to/17D6jfP
http://nbcnews.to/137SoMO
http://nbcnews.to/137S3d1
http://nbcnews.to/109arir
http://nbcnews.to/103DWGS
http://on.today.com/ZrJO88
http://on.today.com/ZrIzWC
http://nbcnews.to/101komt
http://nbcnews.to/11F7vex
http://nbcnews.to/11F6LWZ
http://nbcnews.to/11F0meu
http://nbcnews.to/11EH7Se
http://nbcnews.to/11DkH3N
http://nbcnews.to/11DiQfi
http://nbcnews.to/15akpI7
http://nbcnews.to/15ajh7f
http://nbcnews.to/10t0wTA
http://nbcnews.to/159YJvE
http://nbcnews.to/159Yalo
http://nbcnews.to/YmPFk0
http://nbcnews.to/10sw8sq
http://nbcnews.to/102dTLx
http://nbcnews.to/YlcSTF
http://nbcnews.to/Yc9qbC
http://nbcnews.to/101ugIo
http://nbcnews.to/YbBj3B
http://nbcnews.to/100Tfvh
http://nbcnews.to/YbkeXE
http://nbcnews.to/100fDoH
http://on.today.com/1003rEm
http://nbcnews

http://nbcnews.to/1AAni4j
Allergic to Penicillin? You're Probably Not http://nbcnews.to/1vUq1NV


http://nbcnews.to/1vUq1NV
Viral Baby Announcement Includes Cookies and Screams http://nbcnews.to/1tI6QZp


http://nbcnews.to/1tI6QZp
It's Over: Texas' Ebola Outbreak Has Ended http://nbcnews.to/1vSfBhS


http://nbcnews.to/1vSfBhS
Most People Allergic to Penicillin Are Not: Researchers http://nbcnews.to/1ptV2vC


http://nbcnews.to/1ptV2vC
It's Over: Texas Ebola Outbreak About to End http://nbcnews.to/1xlnRJH


http://nbcnews.to/1xlnRJH
Pokey Robot Creates a Ghostly Presence on Demand http://nbcnews.to/1Guq2Rk


http://nbcnews.to/1Guq2Rk
RT @JuliaSommerfeld: Tales of chucking big jobs are the new lady porn RT @ELLEmagazine: What's REALLY causing women to burn out before 30: �


Is Holiday Coffee Worth the Calories? http://nbcnews.to/10sZNcz


http://nbcnews.to/10sZNcz
A Look Inside US Ebola Facility in Monrovia http://nbcnews.to/1GtvoMp


http://nbcnews.to/1GtvoMp
Ebola Nurse Amber Vinson De

...


http://nbcnews.to/Og2GZ8
School rallies around 510-pound teen to help him get healthy http://on.today.com/1d6Grdq


http://on.today.com/1d6Grdq
EPA Proposes Stronger Pesticide Rules to Protect Workers http://nbcnews.to/1mwICk2


http://nbcnews.to/1mwICk2
Military Uses Unproven Mental Health Programs, Report Finds http://nbcnews.to/1eYODMw


http://nbcnews.to/1eYODMw
Military Uses Unproven Mental Health Screening, Report Finds http://nbcnews.to/1fkrCbf


http://nbcnews.to/1fkrCbf
Flu Killing More Young Adults This Year, CDC Says http://nbcnews.to/1d5HfiN


http://nbcnews.to/1d5HfiN
Accident Rates Improving for Older Drivers http://nbcnews.to/MEmlRh


http://nbcnews.to/MEmlRh
Troubled Vets Get Unproven Treatments, New Report Finds http://nbcnews.to/1h0CPOd


http://nbcnews.to/1h0CPOd
Family Fights to Block Deportation of Comatose Exchange Student  http://nbcnews.to/1ct3H8r


http://nbcnews.to/1ct3H8r
What's It Like to Go Last at the Olympics?  http://nbcnews.to/NduOeo


http://nbcnews.t

http://Healthcare.gov
Depressed people may age faster, study suggests http://nbcnews.to/1eIAp7W


http://nbcnews.to/1eIAp7W
Avoiding the pee splash-back is in the aim, distance, scientist says http://nbcnews.to/1eIjxhC


http://nbcnews.to/1eIjxhC
Being a bully linked to casual, risky sex http://nbcnews.to/1bzMpnk


http://nbcnews.to/1bzMpnk
Medicaid is health overhaul's early success story http://nbcnews.to/1eHua4j


http://nbcnews.to/1eHua4j
Exercise may help prevent depression later in life http://nbcnews.to/1eFnl3b


http://nbcnews.to/1eFnl3b
Vitamins don�t prevent heart disease or cancer,experts find http://nbcnews.to/1eF5Ks9


http://nbcnews.to/1eF5Ks9
Health overhaul debate snags Senate pharmacy-oversight bill http://nbcnews.to/1bvVMUX


http://nbcnews.to/1bvVMUX
Don�t expect big enrollment numbers for the health insurance exchanges http://nbcnews.to/1bvLtjD


http://nbcnews.to/1bvLtjD
Polio strain in Syria originated in Pakistan, WHO confirms http://nbcnews.to/1eECuln


http://n


http://nbcnews.to/1bwb4MO
High Court asked to review 'contraception mandate' http://nbcnews.to/1eW6iut


http://nbcnews.to/1eW6iut
Animals repeatedly infected people with MERS, study suggests http://nbcnews.to/1eW2N7c


http://nbcnews.to/1eW2N7c
VA: Aaron Alexis never sought mental health treatment http://nbcnews.to/1aNPPmP


http://nbcnews.to/1aNPPmP
Texas boy's death reignites debate over EpiPens in schools http://on.today.com/1eqiL8E


http://on.today.com/1eqiL8E
Cat poop parasite controls minds early -- and permanently, study finds http://nbcnews.to/1eqbPbs


http://nbcnews.to/1eqbPbs
E-readers may help dyslexics read more easily http://on.today.com/1dpYw86


http://on.today.com/1dpYw86
Expect bump in health spending next year, federal government forecasts http://nbcnews.to/1dpV1OK


http://nbcnews.to/1dpV1OK
It's never too early for Botox, doctor says http://on.today.com/1dpRlwz


http://on.today.com/1dpRlwz
America's sneeziest cities: 15 worst U.S. cities for fall allergies http



http://nbcnews.to/10Vb0vp
Author admits to faking bladder cancer http://bit.ly/18HvXBz


http://bit.ly/18HvXBz
Angelina Jolie: I have had double mastectomy http://nbcnews.to/10UgFlp


http://nbcnews.to/10UgFlp
CDC: 2012 was deadliest year for West Nile in US http://nbcnews.to/18GaQiV


http://nbcnews.to/18GaQiV
UN urges: Eat more insects! (Seriously) http://nbcnews.to/18G9OUh


http://nbcnews.to/18G9OUh
Some cancer patients may face high drug costs under health law http://nbcnews.to/13euqwO


http://nbcnews.to/13euqwO
Agent Orange liked with aggressive prostate cancer http://nbcnews.to/18FLcuQ


http://nbcnews.to/18FLcuQ
Rumors, grief and questions: A virus ravages a Saudi family http://nbcnews.to/13e3lKj


http://nbcnews.to/13e3lKj
Rare disorder leaves woman lost in familiar places http://on.today.com/YEwKiB


http://on.today.com/YEwKiB
Pediatricians take on gun lobby � carefully http://nbcnews.to/YEk7nC


http://nbcnews.to/YEk7nC
WHO: New SARS-like virus can probably pass person to

RT @nbcnightlynews: CDC: Flu vaccine often doesn�t work. @RobertBazell reports: http://nbcnews.to/15z5cOa


http://nbcnews.to/15z5cOa
Bullies, victims at risk for long-term depression, anxiety http://on.today.com/15z7OeZ


http://on.today.com/15z7OeZ
Red state Medicaid expansion no shock http://nbcnews.to/WdoVw1


http://nbcnews.to/WdoVw1
25 states to start testing new Medicaid models http://nbcnews.to/UNrj02


http://nbcnews.to/UNrj02
More recalls of US-made pet jerky treats, FDA says http://nbcnews.to/YJb5Sh


http://nbcnews.to/YJb5Sh
Flu vaccine barely worked in elderly this year http://nbcnews.to/YJaT5n


http://nbcnews.to/YJaT5n
Feds indict 4 in salmonella outbreak http://nbcnews.to/YIZgeZ


http://nbcnews.to/YIZgeZ
Women deluge hotline in probe of gynecologist http://nbcnews.to/YIMCwz


http://nbcnews.to/YIMCwz
Why young smokers should quit before turning 44 http://on.today.com/Xn1oMs


http://on.today.com/Xn1oMs
Hold the fries: We're eating less fast food http://on.today.com/XmY

In [7]:
frame.head()

,tweetid,date,title,url
0,548617649010532353,Fri Dec 26 23:13:35 +0000 2014,Ebola Exposure?: CDC Worker 'Remains Well',http://nbcnews.to/1tmYMkv
1,548569476795731968,Fri Dec 26 20:02:09 +0000 2014,Pregnant Woman Taken Off Life Support in Ireland,http://nbcnews.to/1BcYvPG
2,548533428023738368,Fri Dec 26 17:38:55 +0000 2014,Money from #IceBucketChallenge still flowing t...,http://ow.ly/GseY3
3,548531135442337793,Fri Dec 26 17:29:48 +0000 2014,Caramel Apples Recalled After Listeria Deaths,http://nbcnews.to/1EqpPjj
4,548194819907284992,Thu Dec 25 19:13:24 +0000 2014,Ice Bucket Challenge Still Rains Bucks on ALS,http://nbcnews.to/1thdMAd


In [8]:
# -------------------------------------------------
frame.loc[:,'title'][0]

"Ebola Exposure?: CDC Worker 'Remains Well' "

In [9]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//article[@class="articleBody"]/div/div[3]/p/text()',
               '',
               ''
               ]

In [10]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [11]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//*[@id="content"]/div/div[6]/div/div/div/article/div/div[3]/div[1]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
#             author = xpath_selector.xpath('//div[@class="info-name"]/text()').extract()
#             if len(author):
#                 author = author[0]
#             else:
#                 author = ''
#             print(len(author))
            list_content.update({'contents': contents,'tags': '','author': ''})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("========================\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://nbcnews.to/1tmYMkv
1197
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  1197
tags  :  0
author  :  0



scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://nbcnews.to/1BcYvPG
1672
contents  :  1672
tags  :  0
author  :  0

Header:   200
http://ow.ly/GseY3
3881
contents  :  3881
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1EqpPjj
1268
contents  :  1268
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1thdMAd
3881
contents  :  3881
tags  :  0
author  :  0

Header:   200
http://ow.ly/GpzmS
3736
contents  :  3736
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1CLABvX
3736
contents  :  3736
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1vbYiYB
4845
contents  :  4845
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1EdtTDx
5374
contents  :  5374
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1zgmtY8
2357
contents  :  2357
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1CDgVKG
1293
contents  :  1293
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1zuy6Qa
1293
contents  :  1293
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13BkCEA
720
co

Header:   200
http://nbcnews.to/1wguqhm
1877
contents  :  1877
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1zLGVlm
5132
contents  :  5132
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/134p9j7
5132
contents  :  5132
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/12jzZAW
4508
contents  :  4508
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/12YHKNu
2282
contents  :  2282
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1yIcuhF
3885
contents  :  3885
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1yt1rFi
4401
contents  :  4401
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1ysGSsy
947
contents  :  947
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/12zJ6NI
138
contents  :  138
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1wcQZ9y
1001
contents  :  1001
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1q7Dy8h
2655
contents  :  2655
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1wb4

Header:   200
http://nbcnews.to/11RouQK
5445
contents  :  5445
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/15eQdgu
3431
contents  :  3431
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1riUpAh
5289
contents  :  5289
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1riACRs
149
contents  :  149
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1v2KyCx
630
contents  :  630
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1y4ro18
3380
contents  :  3380
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/156xmEe
2385
contents  :  2385
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1r9Pv8G
1244
contents  :  1244
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1F9IomV
3888
contents  :  3888
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1xcHthf
1638
contents  :  1638
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1AtKiRn
103
contents  :  103
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1AtKjV

...

Header:   200
http://nbcnews.to/16knlyB
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16klUQQ
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16jZ23I
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13AadYz
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16io1Vg
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16i2X19
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16hSL8N
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/16hOVMR
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://sfy.co/bGHK
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://sfy.co/bGHK
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/16hgqGi
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/16XHV9c
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcne

Header:   200
http://nbcnews.to/15K5scb
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/15JWAmR
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/15JVTtK
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/Y6CIYX
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/Y5tDzx
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/Y5imze
4520
contents  :  4520
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/ZfPJxL
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://nbcnews.to/W1P5WN
Header:   200
http://nbcnews.to/W0ZHoF
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/W0Zimj
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/ZxiOqw
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/WCURiR
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/XdstnY
0
contents  :  0

Header:   200
http://nbcnews.to/X8ZjEd
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/X8ZjnZ
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/XYO1OD
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/ZGfADn
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/ZG50fw
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13LWgRF
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13LuC7z
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13LnmZn
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13LcgmY
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13Lbab1
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/13KOeZt
0
contents  :  0
tags  :  0
author  :  0

['ERRORS'] 404 , at http://nbcnews.to/YvQELO
Header:   200
http://nbcnews.to/13KnNmG
0
contents  :  0
t

Header:   200
http://on.today.com/YUsdGI
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/Z7EU4G
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/VQu50z
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/VQt7Sd
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/VQqbVs
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/VQq0tr
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/Ym1ReH
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/U5C4ua
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/U5zJPW
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/122NK4E
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/122F7ah
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/122znNU
0
contents  :  0
tags  :  0
author  :  0

Header:   200
htt

Header:   200
http://nbcnews.to/XKDiHM
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1113CF7
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/1113DZZ
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://on.today.com/WpYIgv
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/SO4xnL
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/11XwsFP
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/11XuRQd
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/11XphgL
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/XFCfci
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/XFBRdQ
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://www.msnbc.msn.com/id/50587303/ns/health-cold
0
contents  :  0
tags  :  0
author  :  0

Header:   200
http://nbcnews.to/10Ti8xJ
0
contents  :  0
tags  :  0
author 

In [12]:
# -------------------------------------------------
frame.head()

,author,contents,date,tags,title,tweetid,url
0,,"A CDC technician ""remains well"" and continues ...",Fri Dec 26 23:13:35 +0000 2014,,Ebola Exposure?: CDC Worker 'Remains Well',5.486176e+17,http://nbcnews.to/1tmYMkv
1,,A clinically dead pregnant Irishwoman was remo...,Fri Dec 26 20:02:09 +0000 2014,,Pregnant Woman Taken Off Life Support in Ireland,5.485695e+17,http://nbcnews.to/1BcYvPG
2,,The Internet sensation of the year — the Ice B...,Fri Dec 26 17:38:55 +0000 2014,,Money from #IceBucketChallenge still flowing t...,5.485334e+17,http://ow.ly/GseY3
3,,A Missouri company is recalling its Happy Appl...,Fri Dec 26 17:29:48 +0000 2014,,Caramel Apples Recalled After Listeria Deaths,5.485311e+17,http://nbcnews.to/1EqpPjj
4,,The Internet sensation of the year — the Ice B...,Thu Dec 25 19:13:24 +0000 2014,,Ice Bucket Challenge Still Rains Bucks on ALS,5.481948e+17,http://nbcnews.to/1thdMAd


In [13]:
# -------------------------------------------------
frame.shape

(4215, 7)

In [14]:
frame.to_csv(dest_url, sep='|', encoding='utf-8',index=False)

In [2]:
import unicodedata
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
#     success_date = './date/success_date.txt'
    reg_special = r"([^a-zA-Z0-9\s\']+)"
    reg_dspace = r"( {2,})"   # r"(\s{2,})"
    reg_html = r"(<[^>]+>)"
    reg_newlines = r"([\r|\n|\r\n]+)"
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Clean content a bit
            contents = re.sub(reg_html," ",contents)
            contents = unicodedata.normalize('NFKD', contents).encode('ascii', 'ignore').decode('ascii')
            contents = re.sub(reg_special," ", contents)
            contents = re.sub(reg_newlines," ",contents)
            contents = re.sub(reg_dspace," ",contents)
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
            
            #Write success load date
#             f1 = open(success_date, "a")
#             f1.write(frame['date'].iloc[key])
#             f1.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  1197
1  :  1672
2  :  3881
3  :  1268
4  :  3881
5  :  3736
6  :  3736
7  :  4845
8  :  5374
9  :  2357
10  :  1293
11  :  1293
12  :  720
13  :  9196
14  :  3
[Out] key:  14
15  :  139
16  :  5353
17  :  137
18  :  123
19  :  304
20  :  3
[Out] key:  20
21  :  8731
22  :  175
23  :  165
24  :  4111
25  :  7936
26  :  3
[Out] key:  26
27  :  145
28  :  1128
29  :  1254
30  :  1254
31  :  4587
32  :  3144
33  :  3916
34  :  1351
35  :  1046
36  :  4235
37  :  1242
38  :  128
39  :  5272
40  :  3
[Out] key:  40
41  :  115
42  :  143
43  :  142
44  :  2165
45  :  2165
46  :  2165
47  :  1591
48  :  4559
49  :  2446
50  :  2171
51  :  119
52  :  1935
53  :  165
54  :  2395
55  :  188
56  :  128
57  :  149
58  :  145
59  :  8348
60  :  3993
61  :  3
[Out] key:  61
62  :  146
63  :  183
64  :  3303
65  :  2101
66  :  168
67  :  2680
68  :  1363
69  :  1630
70  :  150
71  :  149
72  :  1762
73  :  5074
74  :  5074
75  :  1877
76  :  1877
77  :  5132
78  :  5132
79  :  4508
80  :  2282
8

608  :  147
609  :  2060
610  :  3494
611  :  3
[Out] key:  611
612  :  897
613  :  805
614  :  3260
615  :  122
616  :  911
617  :  3
[Out] key:  617
618  :  951
619  :  146
620  :  4216
621  :  3882
622  :  950
623  :  3
[Out] key:  623
624  :  759
625  :  151
626  :  836
627  :  2520
628  :  3
[Out] key:  628
629  :  965
630  :  5627
631  :  3
[Out] key:  631
632  :  997
633  :  699
634  :  960
635  :  130
636  :  149
637  :  853
638  :  662
639  :  147
640  :  181
641  :  3
[Out] key:  641
642  :  6153
643  :  138
644  :  870
645  :  780
646  :  3
[Out] key:  646
647  :  4916
648  :  128
649  :  150
650  :  3
[Out] key:  650
651  :  3
[Out] key:  651
652  :  3
[Out] key:  652
653  :  816
654  :  4911
655  :  882
656  :  5797
657  :  911
658  :  4964
659  :  139
660  :  662
661  :  140
662  :  1
[Out] key:  662
663  :  3356
664  :  657
665  :  946
666  :  136
667  :  3
[Out] key:  667
668  :  3610
669  :  130
670  :  5175
671  :  5707
672  :  3
[Out] key:  672
673  :  4942
674  :  8

1158  :  81
1159  :  3
[Out] key:  1159
1160  :  145
1161  :  1347
1162  :  4137
1163  :  3
[Out] key:  1163
1164  :  129
1165  :  3
[Out] key:  1165
1166  :  1
[Out] key:  1166
1167  :  1
[Out] key:  1167
1168  :  5628
1169  :  3
[Out] key:  1169
1170  :  3
[Out] key:  1170
1171  :  3
[Out] key:  1171
1172  :  3
[Out] key:  1172
1173  :  125
1174  :  2510
1175  :  4388
1176  :  2159
1177  :  3
[Out] key:  1177
1178  :  3
[Out] key:  1178
1179  :  113
1180  :  3
[Out] key:  1180
1181  :  828
1182  :  2637
1183  :  2482
1184  :  3
[Out] key:  1184
1185  :  5257
1186  :  98
1187  :  5125
1188  :  1914
1189  :  1880
1190  :  139
1191  :  6395
1192  :  138
1193  :  3763
1194  :  2629
1195  :  3
[Out] key:  1195
1196  :  3
[Out] key:  1196
1197  :  138
1198  :  183
1199  :  2177
1200  :  5257
1201  :  183
1202  :  129
1203  :  3
[Out] key:  1203
1204  :  98
1205  :  3
[Out] key:  1205
1206  :  2101
1207  :  2055
1208  :  87
1209  :  3
[Out] key:  1209
1210  :  1098
1211  :  2513
1212  :  48

1559  :  2089
1560  :  3
[Out] key:  1560
1561  :  3202
1562  :  197
1563  :  3
[Out] key:  1563
1564  :  3202
1565  :  5226
1566  :  141
1567  :  5538
1568  :  5319
1569  :  4049
1570  :  1181
1571  :  7112
1572  :  82
1573  :  1809
1574  :  3799
1575  :  136
1576  :  141
1577  :  142
1578  :  3660
1579  :  6538
1580  :  131
1581  :  131
1582  :  3799
1583  :  124
1584  :  118
1585  :  136
1586  :  1932
1587  :  136
1588  :  151
1589  :  2142
1590  :  144
1591  :  1161
1592  :  3588
1593  :  143
1594  :  2337
1595  :  6729
1596  :  115
1597  :  1908
1598  :  2563
1599  :  5397
1600  :  129
1601  :  1864
1602  :  145
1603  :  2377
1604  :  2291
1605  :  716
1606  :  137
1607  :  3438
1608  :  2343
1609  :  2210
1610  :  1547
1611  :  1800
1612  :  1800
1613  :  4869
1614  :  146
1615  :  1818
1616  :  134
1617  :  3
[Out] key:  1617
1618  :  141
1619  :  2419
1620  :  148
1621  :  2419
1622  :  2079
1623  :  2188
1624  :  1853
1625  :  2392
1626  :  126
1627  :  126
1628  :  3750
1629 

2138  :  3
[Out] key:  2138
2139  :  3
[Out] key:  2139
2140  :  3
[Out] key:  2140
2141  :  3
[Out] key:  2141
2142  :  3
[Out] key:  2142
2143  :  5323
2144  :  3
[Out] key:  2144
2145  :  3
[Out] key:  2145
2146  :  3
[Out] key:  2146
2147  :  3
[Out] key:  2147
2148  :  3
[Out] key:  2148
2149  :  3
[Out] key:  2149
2150  :  3
[Out] key:  2150
2151  :  2239
2152  :  3
[Out] key:  2152
2153  :  3
[Out] key:  2153
2154  :  3
[Out] key:  2154
2155  :  3
[Out] key:  2155
2156  :  3
[Out] key:  2156
2157  :  3
[Out] key:  2157
2158  :  3
[Out] key:  2158
2159  :  3
[Out] key:  2159
2160  :  3
[Out] key:  2160
2161  :  3497
2162  :  3
[Out] key:  2162
2163  :  3
[Out] key:  2163
2164  :  3
[Out] key:  2164
2165  :  3
[Out] key:  2165
2166  :  3
[Out] key:  2166
2167  :  3
[Out] key:  2167
2168  :  3
[Out] key:  2168
2169  :  3
[Out] key:  2169
2170  :  3
[Out] key:  2170
2171  :  3
[Out] key:  2171
2172  :  3
[Out] key:  2172
2173  :  3
[Out] key:  2173
2174  :  3
[Out] key:  2174
2175  

[Out] key:  2514
2515  :  3
[Out] key:  2515
2516  :  3
[Out] key:  2516
2517  :  3
[Out] key:  2517
2518  :  3
[Out] key:  2518
2519  :  3
[Out] key:  2519
2520  :  3
[Out] key:  2520
2521  :  3
[Out] key:  2521
2522  :  3
[Out] key:  2522
2523  :  3
[Out] key:  2523
2524  :  3
[Out] key:  2524
2525  :  3
[Out] key:  2525
2526  :  3
[Out] key:  2526
2527  :  3
[Out] key:  2527
2528  :  3
[Out] key:  2528
2529  :  3
[Out] key:  2529
2530  :  3
[Out] key:  2530
2531  :  3
[Out] key:  2531
2532  :  3
[Out] key:  2532
2533  :  3
[Out] key:  2533
2534  :  3
[Out] key:  2534
2535  :  3
[Out] key:  2535
2536  :  3
[Out] key:  2536
2537  :  3
[Out] key:  2537
2538  :  3
[Out] key:  2538
2539  :  3
[Out] key:  2539
2540  :  3
[Out] key:  2540
2541  :  3
[Out] key:  2541
2542  :  3
[Out] key:  2542
2543  :  3
[Out] key:  2543
2544  :  3
[Out] key:  2544
2545  :  3
[Out] key:  2545
2546  :  3
[Out] key:  2546
2547  :  3
[Out] key:  2547
2548  :  3
[Out] key:  2548
2549  :  3
[Out] key:  2549
255

2978  :  2038
2979  :  4397
2980  :  786
2981  :  3
[Out] key:  2981
2982  :  5731
2983  :  3
[Out] key:  2983
2984  :  882
2985  :  5500
2986  :  1216
2987  :  7273
2988  :  3561
2989  :  4901
2990  :  5823
2991  :  4376
2992  :  4911
2993  :  2636
2994  :  4377
2995  :  3159
2996  :  2411
2997  :  4885
2998  :  3055
2999  :  4898
3000  :  5920
3001  :  5560
3002  :  1904
3003  :  2553
3004  :  5866
3005  :  4772
3006  :  3166
3007  :  3
[Out] key:  3007
3008  :  4183
3009  :  3144
3010  :  3228
3011  :  4870
3012  :  3
[Out] key:  3012
3013  :  3697
3014  :  7195
3015  :  2598
3016  :  7519
3017  :  3
[Out] key:  3017
3018  :  3826
3019  :  3
[Out] key:  3019
3020  :  4313
3021  :  3
[Out] key:  3021
3022  :  4001
3023  :  5556
3024  :  3902
3025  :  2829
3026  :  4193
3027  :  4151
3028  :  1452
3029  :  3013
3030  :  837
3031  :  3786
3032  :  4326
3033  :  3642
3034  :  3786
3035  :  3
[Out] key:  3035
3036  :  4501
3037  :  3958
3038  :  3
[Out] key:  3038
3039  :  8533
3040  :  

[Out] key:  3531
3532  :  3
[Out] key:  3532
3533  :  3
[Out] key:  3533
3534  :  3
[Out] key:  3534
3535  :  3
[Out] key:  3535
3536  :  3
[Out] key:  3536
3537  :  3
[Out] key:  3537
3538  :  3
[Out] key:  3538
3539  :  3
[Out] key:  3539
3540  :  3
[Out] key:  3540
3541  :  3
[Out] key:  3541
3542  :  4075
3543  :  3
[Out] key:  3543
3544  :  3
[Out] key:  3544
3545  :  3
[Out] key:  3545
3546  :  3
[Out] key:  3546
3547  :  3
[Out] key:  3547
3548  :  3
[Out] key:  3548
3549  :  3
[Out] key:  3549
3550  :  3
[Out] key:  3550
3551  :  3
[Out] key:  3551
3552  :  3
[Out] key:  3552
3553  :  3
[Out] key:  3553
3554  :  3
[Out] key:  3554
3555  :  3
[Out] key:  3555
3556  :  3
[Out] key:  3556
3557  :  3
[Out] key:  3557
3558  :  3
[Out] key:  3558
3559  :  3
[Out] key:  3559
3560  :  3
[Out] key:  3560
3561  :  3
[Out] key:  3561
3562  :  3
[Out] key:  3562
3563  :  3
[Out] key:  3563
3564  :  3
[Out] key:  3564
3565  :  3
[Out] key:  3565
3566  :  3
[Out] key:  3566
3567  :  3
[Out] 

3834  :  3
[Out] key:  3834
3835  :  3
[Out] key:  3835
3836  :  3
[Out] key:  3836
3837  :  3
[Out] key:  3837
3838  :  3
[Out] key:  3838
3839  :  3
[Out] key:  3839
3840  :  3
[Out] key:  3840
3841  :  3
[Out] key:  3841
3842  :  3
[Out] key:  3842
3843  :  2623
3844  :  3
[Out] key:  3844
3845  :  3
[Out] key:  3845
3846  :  3
[Out] key:  3846
3847  :  3
[Out] key:  3847
3848  :  3
[Out] key:  3848
3849  :  3
[Out] key:  3849
3850  :  3
[Out] key:  3850
3851  :  3
[Out] key:  3851
3852  :  3
[Out] key:  3852
3853  :  3
[Out] key:  3853
3854  :  3
[Out] key:  3854
3855  :  3
[Out] key:  3855
3856  :  3
[Out] key:  3856
3857  :  3
[Out] key:  3857
3858  :  3
[Out] key:  3858
3859  :  3
[Out] key:  3859
3860  :  3
[Out] key:  3860
3861  :  3
[Out] key:  3861
3862  :  3
[Out] key:  3862
3863  :  3
[Out] key:  3863
3864  :  3
[Out] key:  3864
3865  :  3
[Out] key:  3865
3866  :  3
[Out] key:  3866
3867  :  3
[Out] key:  3867
3868  :  3
[Out] key:  3868
3869  :  3
[Out] key:  3869
3870  

In [21]:
# If End of jupyter-notebook, reset the "frame" Dataframe as 4 columns
### Writing content into text file
### ================================
file_names = frame['date'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './date/date.txt'
    
    try:
    
        # Writing on file
        contents = frame['date'].iloc[key]
        print(key, ' : ' ,len(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"a")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  30
1  :  30
2  :  30
3  :  30
4  :  30
5  :  30
6  :  30
7  :  30
8  :  30
9  :  30
10  :  30
11  :  30
12  :  30
13  :  30
14  :  30
15  :  30
16  :  30
17  :  30
18  :  30
19  :  30
20  :  30
21  :  30
22  :  30
23  :  30
24  :  30
25  :  30
26  :  30
27  :  30
28  :  30
29  :  30
30  :  30
31  :  30
32  :  30
33  :  30
34  :  30
35  :  30
36  :  30
37  :  30
38  :  30
39  :  30
40  :  30
41  :  30
42  :  30
43  :  30
44  :  30
45  :  30
46  :  30
47  :  30
48  :  30
49  :  30
50  :  30
51  :  30
52  :  30
53  :  30
54  :  30
55  :  30
56  :  30
57  :  30
58  :  30
59  :  30
60  :  30
61  :  30
62  :  30
63  :  30
64  :  30
65  :  30
66  :  30
67  :  30
68  :  30
69  :  30
70  :  30
71  :  30
72  :  30
73  :  30
74  :  30
75  :  30
76  :  30
77  :  30
78  :  30
79  :  30
80  :  30
81  :  30
82  :  30
83  :  30
84  :  30
85  :  30
86  :  30
87  :  30
88  :  30
89  :  30
90  :  30
91  :  30
92  :  30
93  :  30
94  :  30
95  :  30
96  :  30
97  :  30
98  :  30
99  :  30
100  :  30

1026  :  30
1027  :  30
1028  :  30
1029  :  30
1030  :  30
1031  :  30
1032  :  30
1033  :  30
1034  :  30
1035  :  30
1036  :  30
1037  :  30
1038  :  30
1039  :  30
1040  :  30
1041  :  30
1042  :  30
1043  :  30
1044  :  30
1045  :  30
1046  :  30
1047  :  30
1048  :  30
1049  :  30
1050  :  30
1051  :  30
1052  :  30
1053  :  30
1054  :  30
1055  :  30
1056  :  30
1057  :  30
1058  :  30
1059  :  30
1060  :  30
1061  :  30
1062  :  30
1063  :  30
1064  :  30
1065  :  30
1066  :  30
1067  :  30
1068  :  30
1069  :  30
1070  :  30
1071  :  30
1072  :  30
1073  :  30
1074  :  30
1075  :  30
1076  :  30
1077  :  30
1078  :  30
1079  :  30
1080  :  30
1081  :  30
1082  :  30
1083  :  30
1084  :  30
1085  :  30
1086  :  30
1087  :  30
1088  :  30
1089  :  30
1090  :  30
1091  :  30
1092  :  30
1093  :  30
1094  :  30
1095  :  30
1096  :  30
1097  :  30
1098  :  30
1099  :  30
1100  :  30
1101  :  30
1102  :  30
1103  :  30
1104  :  30
1105  :  30
1106  :  30
1107  :  30
1108  :  30
1109

2018  :  30
2019  :  30
2020  :  30
2021  :  30
2022  :  30
2023  :  30
2024  :  30
2025  :  30
2026  :  30
2027  :  30
2028  :  30
2029  :  30
2030  :  30
2031  :  30
2032  :  30
2033  :  30
2034  :  30
2035  :  30
2036  :  30
2037  :  30
2038  :  30
2039  :  30
2040  :  30
2041  :  30
2042  :  30
2043  :  30
2044  :  30
2045  :  30
2046  :  30
2047  :  30
2048  :  30
2049  :  30
2050  :  30
2051  :  30
2052  :  30
2053  :  30
2054  :  30
2055  :  30
2056  :  30
2057  :  30
2058  :  30
2059  :  30
2060  :  30
2061  :  30
2062  :  30
2063  :  30
2064  :  30
2065  :  30
2066  :  30
2067  :  30
2068  :  30
2069  :  30
2070  :  30
2071  :  30
2072  :  30
2073  :  30
2074  :  30
2075  :  30
2076  :  30
2077  :  30
2078  :  30
2079  :  30
2080  :  30
2081  :  30
2082  :  30
2083  :  30
2084  :  30
2085  :  30
2086  :  30
2087  :  30
2088  :  30
2089  :  30
2090  :  30
2091  :  30
2092  :  30
2093  :  30
2094  :  30
2095  :  30
2096  :  30
2097  :  30
2098  :  30
2099  :  30
2100  :  30
2101

2966  :  30
2967  :  30
2968  :  30
2969  :  30
2970  :  30
2971  :  30
2972  :  30
2973  :  30
2974  :  30
2975  :  30
2976  :  30
2977  :  30
2978  :  30
2979  :  30
2980  :  30
2981  :  30
2982  :  30
2983  :  30
2984  :  30
2985  :  30
2986  :  30
2987  :  30
2988  :  30
2989  :  30
2990  :  30
2991  :  30
2992  :  30
2993  :  30
2994  :  30
2995  :  30
2996  :  30
2997  :  30
2998  :  30
2999  :  30
3000  :  30
3001  :  30
3002  :  30
3003  :  30
3004  :  30
3005  :  30
3006  :  30
3007  :  30
3008  :  30
3009  :  30
3010  :  30
3011  :  30
3012  :  30
3013  :  30
3014  :  30
3015  :  30
3016  :  30
3017  :  30
3018  :  30
3019  :  30
3020  :  30
3021  :  30
3022  :  30
3023  :  30
3024  :  30
3025  :  30
3026  :  30
3027  :  30
3028  :  30
3029  :  30
3030  :  30
3031  :  30
3032  :  30
3033  :  30
3034  :  30
3035  :  30
3036  :  30
3037  :  30
3038  :  30
3039  :  30
3040  :  30
3041  :  30
3042  :  30
3043  :  30
3044  :  30
3045  :  30
3046  :  30
3047  :  30
3048  :  30
3049

3773  :  30
3774  :  30
3775  :  30
3776  :  30
3777  :  30
3778  :  30
3779  :  30
3780  :  30
3781  :  30
3782  :  30
3783  :  30
3784  :  30
3785  :  30
3786  :  30
3787  :  30
3788  :  30
3789  :  30
3790  :  30
3791  :  30
3792  :  30
3793  :  30
3794  :  30
3795  :  30
3796  :  30
3797  :  30
3798  :  30
3799  :  30
3800  :  30
3801  :  30
3802  :  30
3803  :  30
3804  :  30
3805  :  30
3806  :  30
3807  :  30
3808  :  30
3809  :  30
3810  :  30
3811  :  30
3812  :  30
3813  :  30
3814  :  30
3815  :  30
3816  :  30
3817  :  30
3818  :  30
3819  :  30
3820  :  30
3821  :  30
3822  :  30
3823  :  30
3824  :  30
3825  :  30
3826  :  30
3827  :  30
3828  :  30
3829  :  30
3830  :  30
3831  :  30
3832  :  30
3833  :  30
3834  :  30
3835  :  30
3836  :  30
3837  :  30
3838  :  30
3839  :  30
3840  :  30
3841  :  30
3842  :  30
3843  :  30
3844  :  30
3845  :  30
3846  :  30
3847  :  30
3848  :  30
3849  :  30
3850  :  30
3851  :  30
3852  :  30
3853  :  30
3854  :  30
3855  :  30
3856